In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../../../')

from notebooks.utils import load_node_features, get_referral_sites_edges, export_model_as_feature
from train import run_experiment

2022-02-26 23:53:18.391278: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load referral sites edges for level 1

In [2]:
level = 1
referral_sites_NODES = get_referral_sites_edges(data_year=2020, level=level)

print(referral_sites_NODES[:5])

02-26 23:53:18 notebooks.utils INFO     Processing level 0
02-26 23:53:18 notebooks.utils INFO     Node length: 3258
02-26 23:53:18 notebooks.utils INFO     Distinct node length: 3258
02-26 23:53:18 notebooks.utils INFO     Processing level 1
02-26 23:53:18 notebooks.utils INFO     Node length: 9178
02-26 23:53:18 notebooks.utils INFO     Distinct node length: 9178


[('stonecoldtruth.com', 'patriots4truth.org'), ('newnation.org', 'dailykenn.blogspot.com'), ('thenewcivilrightsmovement.com', 'joemygod.com'), ('imprimis.hillsdale.edu', 'winstonchurchill.org'), ('electronicintifada.net', 'mondoweiss.net')]


In [3]:
edge_df = pd.DataFrame(referral_sites_NODES, columns=['source', 'target'])

edge_df.head()

,source,target
0,stonecoldtruth.com,patriots4truth.org
1,newnation.org,dailykenn.blogspot.com
2,thenewcivilrightsmovement.com,joemygod.com
3,imprimis.hillsdale.edu,winstonchurchill.org
4,electronicintifada.net,mondoweiss.net


### Find all unique nodes in edges

In [4]:
nodes_in_edges = list(set(edge_df.source.unique().tolist() + edge_df.target.unique().tolist()))
print('Number of unique nodes in edges:', len(nodes_in_edges), 'Sample:', nodes_in_edges[:5])

Number of unique nodes in edges: 6927 Sample: ['journalism.org', 'psychjobsearch.wikidot.com', 'iowahouserepublicans.com', 'aripaev.ee', 'mymanatee.org']


### 1. Load all node features

In [5]:
node_features_df = load_node_features()
node_features_df = node_features_df.set_index('site')
node_features_df.head()

,alexa_rank,daily_pageviews_per_visitor,daily_time_on_site,total_sites_linking_in,bounce_rate
site,,,,,
whistleblowersandrelators.com,NaN,NaN,NaN,NaN,NaN
geokov.com,2238341.0,1.0,NaN,60.0,0.900
trainingandfacilitation.ca,NaN,NaN,NaN,NaN,NaN
plumsolutions.com.au,1023533.0,1.0,138.0,60.0,0.813
dbdailyupdate.com,145283.0,1.7,179.0,64.0,0.756


# Subset node_features

In [6]:
node_features_df = node_features_df.loc[nodes_in_edges]

In [7]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6927 entries, journalism.org to meforum.org
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   5467 non-null   float64
 1   daily_pageviews_per_visitor  5467 non-null   float64
 2   daily_time_on_site           4197 non-null   float64
 3   total_sites_linking_in       6826 non-null   float64
 4   bounce_rate                  3913 non-null   float64
dtypes: float64(5)
memory usage: 324.7+ KB


### 2. Fill all missing alexa_rank and total_sites_linking_in with 0 

In [8]:
node_features_df.alexa_rank = node_features_df.alexa_rank.fillna(0)
node_features_df.total_sites_linking_in = node_features_df.total_sites_linking_in.fillna(0)
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6927 entries, journalism.org to meforum.org
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   6927 non-null   float64
 1   daily_pageviews_per_visitor  5467 non-null   float64
 2   daily_time_on_site           4197 non-null   float64
 3   total_sites_linking_in       6927 non-null   float64
 4   bounce_rate                  3913 non-null   float64
dtypes: float64(5)
memory usage: 324.7+ KB


### 3. Normalizing features

In [9]:
import math

node_features_df['normalized_alexa_rank'] = node_features_df['alexa_rank'].apply(lambda x: 1/x if x else 0)
node_features_df['normalized_total_sites_linked_in'] = node_features_df['total_sites_linking_in'].apply(lambda x: math.log2(x) if x else 0)

# Create Graph

In [10]:
import stellargraph as sg

G = sg.StellarGraph(nodes=node_features_df.loc[nodes_in_edges, ['normalized_alexa_rank', 'normalized_total_sites_linked_in']], edges=edge_df)

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 6927, Edges: 12436

 Node types:
  default: [6927]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [12436]
        Weights: all 1 (default)
        Features: none


# Unsupervised Attrib2Vec

In [11]:
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

# 1. Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk, and random seed.

nodes = list(G.nodes())
number_of_walks = 1
length = 5

# 2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.
unsupervised_samples = UnsupervisedSampler(G, nodes=nodes, length=length, number_of_walks=number_of_walks)

# 3. Create a node pair generator:
batch_size = 50
epochs = 4
num_samples = [10, 5]

generator = Attri2VecLinkGenerator(G, batch_size)
train_gen = generator.flow(unsupervised_samples)

layer_sizes = [128]
attri2vec = Attri2Vec(layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None)

# Build the model and expose input and output sockets of attri2vec, for node pair inputs:
x_inp, x_out = attri2vec.in_out_tensors()

prediction = link_classification(output_dim=1, output_act="sigmoid", edge_embedding_method="ip")(x_out)

model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [12]:
history = model.fit(train_gen, epochs=32, verbose=2, use_multiprocessing=False, workers=1, shuffle=True)

Epoch 1/32
1109/1109 - 9s - loss: 0.6969 - binary_accuracy: 0.5309 - 9s/epoch - 8ms/step
Epoch 2/32
1109/1109 - 8s - loss: 0.6692 - binary_accuracy: 0.5450 - 8s/epoch - 7ms/step
Epoch 3/32
1109/1109 - 9s - loss: 0.6595 - binary_accuracy: 0.5435 - 9s/epoch - 8ms/step
Epoch 4/32
1109/1109 - 8s - loss: 0.6536 - binary_accuracy: 0.5458 - 8s/epoch - 7ms/step
Epoch 5/32
1109/1109 - 8s - loss: 0.6517 - binary_accuracy: 0.5457 - 8s/epoch - 7ms/step
Epoch 6/32
1109/1109 - 8s - loss: 0.6509 - binary_accuracy: 0.5441 - 8s/epoch - 7ms/step
Epoch 7/32
1109/1109 - 8s - loss: 0.6495 - binary_accuracy: 0.5431 - 8s/epoch - 7ms/step
Epoch 8/32
1109/1109 - 8s - loss: 0.6476 - binary_accuracy: 0.5455 - 8s/epoch - 7ms/step
Epoch 9/32
1109/1109 - 8s - loss: 0.6481 - binary_accuracy: 0.5489 - 8s/epoch - 7ms/step
Epoch 10/32
1109/1109 - 8s - loss: 0.6442 - binary_accuracy: 0.5507 - 8s/epoch - 7ms/step
Epoch 11/32
1109/1109 - 8s - loss: 0.6415 - binary_accuracy: 0.5523 - 8s/epoch - 8ms/step
Epoch 12/32
1109/11

In [13]:
x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

node_gen = Attri2VecNodeGenerator(G, batch_size).flow(node_features_df.index.tolist())
node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=1)

embeddings_wv = dict(zip(node_features_df.index.tolist(), node_embeddings.tolist()))

print('Sample:', embeddings_wv['crooked.com'][:10], len(embeddings_wv['crooked.com']))

139/139 [==============================] - 0s 967us/step
Sample: [0.0014454424381256104, 0.0002391338348388672, 0.004812151193618774, 6.138326625659829e-06, 0.058870404958724976, 0.0015165209770202637, 0.04176586866378784, 0.005881279706954956, 0.05572184920310974, 0.004107058048248291] 128


# Export embeddings as feature

In [14]:
export_model_as_feature(embeddings_wv, f'attrib2vec_referral_sites_level_{level}_epochs_{epochs}')

'/Users/panayot/Documents/News-Media-Peers/data/acl2020/features/attrib2vec_referral_sites_level_1_epochs_4.json'

In [15]:
run_experiment(features=f'attrib2vec_referral_sites_level_{level}_epochs_{epochs}')

+------+---------+---------------------+---------------+--------------------+--------------------------------------------+
| task | dataset | classification_mode | type_training | normalize_features |                  features                  |
+------+---------+---------------------+---------------+--------------------+--------------------------------------------+
| fact | acl2020 |  single classifier  |    combine    |       False        | attrib2vec_referral_sites_level_1_epochs_4 |
+------+---------+---------------------+---------------+--------------------+--------------------------------------------+


02-26 23:58:06 train        INFO     Start training...
02-26 23:58:06 train        INFO     Fold: 0
02-26 23:58:12 train        INFO     Fold: 1
02-26 23:58:18 train        INFO     Fold: 2
02-26 23:58:22 train        INFO     Fold: 3
02-26 23:58:26 train        INFO     Fold: 4


+------+---------+---------------------+---------------+--------------------+--------------------------------------------+-------------------+-------------------+--------------------+--------------------+
| task | dataset | classification_mode | type_training | normalize_features |                  features                  |      Macro-F1     |      Accuracy     |  Flip error-rate   |        MAE         |
+------+---------+---------------------+---------------+--------------------+--------------------------------------------+-------------------+-------------------+--------------------+--------------------+
| fact | acl2020 |  single classifier  |    combine    |       False        | attrib2vec_referral_sites_level_1_epochs_4 | 35.10440172331013 | 54.59837019790454 | 15.133876600698487 | 0.6053550640279395 |
+------+---------+---------------------+---------------+--------------------+--------------------------------------------+-------------------+-------------------+------------------

In [16]:
run_experiment(features=f'attrib2vec_referral_sites_level_{level}_epochs_{epochs}', task='bias')

+------+---------+---------------------+---------------+--------------------+--------------------------------------------+
| task | dataset | classification_mode | type_training | normalize_features |                  features                  |
+------+---------+---------------------+---------------+--------------------+--------------------------------------------+
| bias | acl2020 |  single classifier  |    combine    |       False        | attrib2vec_referral_sites_level_1_epochs_4 |
+------+---------+---------------------+---------------+--------------------+--------------------------------------------+


02-26 23:58:31 train        INFO     Start training...
02-26 23:58:31 train        INFO     Fold: 0
02-26 23:58:36 train        INFO     Fold: 1
02-26 23:58:41 train        INFO     Fold: 2
02-26 23:58:46 train        INFO     Fold: 3
02-26 23:58:50 train        INFO     Fold: 4


+------+---------+---------------------+---------------+--------------------+--------------------------------------------+--------------------+-------------------+--------------------+--------------------+
| task | dataset | classification_mode | type_training | normalize_features |                  features                  |      Macro-F1      |      Accuracy     |  Flip error-rate   |        MAE         |
+------+---------+---------------------+---------------+--------------------+--------------------------------------------+--------------------+-------------------+--------------------+--------------------+
| bias | acl2020 |  single classifier  |    combine    |       False        | attrib2vec_referral_sites_level_1_epochs_4 | 35.441893292750166 | 45.75087310826543 | 13.038416763678695 | 0.6728754365541327 |
+------+---------+---------------------+---------------+--------------------+--------------------------------------------+--------------------+-------------------+-------------